In [2]:
import sqlite3
import requests

### Create the database

In [ ]:
import sqlite3

conn = sqlite3.connect(
    r"C:\Users\Toshiba\Documents\PD2_john\databases\SensorReadings.db"
)
cursor = conn.cursor()

# Create the SensorReadings table
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS SensorReadings (
        ID INTEGER PRIMARY KEY,
        UserID INTEGER NOT NULL,
        Therm TEXT,
        ECG TEXT,
        Airflow TEXT,
        Snore TEXT,
        SpO2 TEXT,
        HR TEXT,
        TimeIn DATETIME,
        TimeOut DATETIME
    )
"""
)

# Commit the changes and close the connection
conn.commit()
conn.close()

### Create artificial data

In [1]:
import random
import string
from datetime import datetime, timedelta

start_time = datetime(2023, 6, 12, 8, 0, 0)

In [16]:
import random
import string
from datetime import datetime, timedelta

# Generate a random 6-digit userID using letters and digits
userID = "".join(random.choices(string.ascii_letters + string.digits, k=6))

# Define the start time and end time for TimeIn and TimeOut
start_time = datetime(2023, 6, 12, 8, 0, 0)
end_time = start_time + timedelta(hours=8)

# Define the number of data points
num_data_points = 691200

# Convert lists to strings
therm_data = str([random.randint(0, 500) for _ in range(num_data_points)])
ecg_data = str([random.randint(0, 500) for _ in range(num_data_points)])
airflow_data = str([random.randint(0, 500) for _ in range(num_data_points)])
snore_data = str([random.randint(0, 500) for _ in range(num_data_points)])
spo2_data = str([random.randint(0, 500) for _ in range(num_data_points)])
hr_data = str([random.randint(0, 500) for _ in range(num_data_points)])

# Create the dictionary with keys and values
data_dict = {
    "UserID": userID,
    "Therm": therm_data,
    "ECG": ecg_data,
    "Airflow": airflow_data,
    "Snore": snore_data,
    "SpO2": spo2_data,
    "HR": hr_data,
    "TimeIn": start_time.strftime("%Y-%m-%d %H:%M:%S"),
    "TimeOut": end_time.strftime("%Y-%m-%d %H:%M:%S"),
}

In [17]:
data_dict["TimeIn"]

'2023-06-12 08:00:00'

### Insert values into the database

In [19]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(
    r"C:\Users\Toshiba\Documents\PD2_john\databases\SensorReadings.db"
)
cursor = conn.cursor()

# Insert the data into the table
cursor.execute(
    """
    INSERT INTO SensorReadings (UserID, Therm, ECG, Airflow, Snore, SpO2, HR, TimeIn, TimeOut)
    VALUES (:UserID, :Therm, :ECG, :Airflow, :Snore, :SpO2, :HR, :TimeIn, :TimeOut)
""",
    data_dict,
)

# Commit the transaction
conn.commit()

# Close the connection
conn.close()

### Retrieving Data

In [21]:
conn = sqlite3.connect(
    r"C:\Users\Toshiba\Documents\PD2_john\databases\SensorReadings.db"
)
cursor = conn.cursor()


# Function to retrieve a specific row based on UserID
def retrieve_row_by_userID(userID):
    # Execute the SELECT statement with a WHERE clause
    cursor.execute("SELECT * FROM SensorReadings WHERE UserID=?", (userID,))

    # Fetch the row from the result
    row = cursor.fetchone()

    if row is not None:
        # Extract the values from the row
        id_, userID, Therm, ECG, Airflow, Snore, SpO2, HR, TimeIn, TimeOut = row

        # Create a dictionary to store the retrieved data
        data_dict2 = {
            "UserID": userID,
            "Therm": Therm,
            "ECG": ECG,
            "Airflow": Airflow,
            "Snore": Snore,
            "SpO2": SpO2,
            "HR": HR,
            "TimeIn": TimeIn,
            "TimeOut": TimeOut,
        }

        return data_dict2

    else:
        return "row data does not exist"


# Retrieve a specific row based on the provided UserID
input_userID = "OYKKRb"
retrieved_data = retrieve_row_by_userID(input_userID)

# Process the retrieved data
if retrieved_data is not None:
    print("Retrieved Data Successfully")
else:
    print("No data found for the provided UserID.")

# Close the connection
conn.close()

Retrieved Data Successfully


### Turn Sensor Data from Str to Lists

In [40]:
import ast

In [37]:
therm_data = ast.literal_eval(retrieved_data["Therm"])

In [39]:
type(therm_data), len(therm_data)

(list, 691200)

### Sending to the Backend to save something in the database

Do a post request and see if it will go through

In [41]:
import requests

In [46]:
url = "http://127.0.0.1:5000/data"  # Replace with your backend URL

data = {"color": "red"}  # Replace with your data

response = requests.post(url, json=data)
print(response.status_code, response.reason, response.text)

200 OK Data received and processed


Send Data to the database using expected sensor readings

In [49]:
data_dict_2 = data_dict.copy()

In [50]:
data_dict_2["UserID"] = "ACC123"
data_dict_2["UserID"]

'ACC123'

In [8]:
url = "http://127.0.0.1:5000/insert"  # Replace with your backend URL

data = data_dict_2  # Replace with your data

response = requests.post(url, json=data)
print(response.status_code, response.reason, response.text)

200 OK


### Retrieve data from using backend

In [30]:
url = "http://localhost:5000/retrieve"  # Replace with your backend URL

data = {"UserID": "ACC123"}  # Replace with your data

response = requests.post(url, json=data)

if response.status_code == 200:
    data_dict2 = response.json()
    print(response.status_code, response.reason)
else:
    print("Error:", response.status_code, response.reason, response.text)

Error: 404 NOT FOUND row does not exist


In [28]:
data_dict2["Therm"][0:100]

'[224, 127, 426, 166, 456, 488, 432, 420, 415, 231, 290, 202, 28, 171, 490, 287, 411, 246, 87, 134, 7'